### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [3]:
#a)
baby_df['review']=baby_df['review'].astype(str) 
baby_df["review"] = baby_df["review"].apply(remove_punctuation)

#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [4]:
#b)
baby_df["review"].replace(np.nan, "", inplace=True)

#short test:
baby_df["review"][38] == baby_df["review"][38]

True

In [5]:
#c)
baby_df = baby_df[baby_df.rating != 3]
#short test:
sum(baby_df["rating"] == 3)

0

In [6]:
#d)

baby_df["rating"].replace(2, -1, inplace=True)
baby_df["rating"].replace(1, -1, inplace=True)
baby_df["rating"].replace(4, 1, inplace=True)
baby_df["rating"].replace(5, 1, inplace=True)
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [9]:
#a)
from sklearn.model_selection import train_test_split

train , test = train_test_split(baby_df, train_size=0.8, test_size=0.2)


In [10]:
#b)
vectorizer = CountVectorizer()

x_train = vectorizer.fit_transform(list(train ["review"]))
y_train = train ["rating"]
x_test = vectorizer.transform(list(test["review"]))
y_test = test["rating"]


## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [11]:
#a)

model = LogisticRegression()
model.fit(x_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
#b)
z_coef = list(zip(list(range(model.coef_.shape[1])), model.coef_[0]))
sort = sorted(z_coef, key= lambda v: v[1])
sort = np.array(sort)
index = sort[:, 0].astype(int)

words = np.array(vectorizer.get_feature_names())
pos = words[index[-10:]]
neg = words[index[:10]]

print("Positive words: ", pos)
print("Negative words: ", neg)
#hint: model.coef_, vectorizer.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Positive words:  ['satisfied' 'negative' 'glad' 'complaint' 'perfect' 'amazing' 'worry'
 'pleased' 'awesome' 'excellent']
Negative words:  ['worst' 'disappointing' 'concept' 'poorly' 'useless' 'returning' 'poor'
 'disappointed' 'horrible' 'terrible']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [13]:
#a)
pred = model.predict(x_test)
print(pred)

[ 1 -1  1 ...  1  1  1]


In [14]:
#b)
pred_2 = model.predict_proba(x_test)
print(pred_2)
#hint: model.predict_proba()

[[1.18958820e-03 9.98810412e-01]
 [8.40721938e-01 1.59278062e-01]
 [1.60387935e-01 8.39612065e-01]
 ...
 [1.13008090e-01 8.86991910e-01]
 [2.32190968e-05 9.99976781e-01]
 [8.21990944e-03 9.91780091e-01]]


In [15]:
#c) 
print("Positive reviews: ", test.iloc[pred_2.T[1].argmax()]["review"])
print("Negative: reviews", test.iloc[pred_2.T[0].argmax()]["review"])
#hint: use the results of b)

Positive reviews:  These plates are amazing  First they are made from Potatoes  Almost all platescupsutensils found on store shelves today are made of plastic  Dont get me wrong I do appreciate plastic and I use it on a daily basis  However it is amazing that there is a way to create childrens products from a biodegradable safe efficient and all natural source  It makes me feel better knowing my kids are eating off a plate that is not leaching any chemicals into their food  I also love that this product will not just sit in the landfill for hundreds of years but will biodegrade faster than other plastic plates So these products by Spuds are not only good for my kidsbut good for the environmentI LOVE thatNow onto the plate itself  I love the design  It is perfect for my one year old who is starting to eat a wider variety of solid foods  Each section is just the right size to put a little bit of food into  A diced strawberry in one section some applesauce in another a few puffs in anothe

In [16]:
#d) 
model.score(x_test, y_test)


0.9318461215555756

## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [17]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [19]:
#a)
vectorizer_2 = CountVectorizer(vocabulary=significant_words)

x_train_2 = vectorizer_2.fit_transform(list(train ["review"]))
y_train_2 = train["rating"]
x_test_2 = vectorizer_2.transform(list(test["review"]))
y_test_2 = test["rating"]

model_2 = LogisticRegression()
model_2.fit(x_train_2, y_train_2)
predict_2 = model_2.predict_proba(x_test_2)
print("Positive reviews: ", test.iloc[predict_2.T[1].argmax()]["review"])
print("Negative: reviews ", test.iloc[predict_2.T[0].argmax()]["review"])
model_2.score(x_test_2, y_test_2)

Positive reviews:  I love this double stroller for many reasons My 9 month olds are very squirmy but I couldnt stand the SUV double that we owned before I love the swiftAt BRU we test drove a few strollers after returning our borowed DuoGlider to the original owner more of a sigh of relief than a sigh of angerWell we tried the DuoRider This stroller was completely inferior to any double I had ever laid eyes on The straps were flimsy lapbelts there was minimal storage only ONE canopy to cover both children plus I cant imagine an average 3 year old fitting in it it was too large yet the seats were too cramped The wheels were flimsy and squeaky it was just horribleI saw the double Jeep stroller it was definatly an umbrella stroller It was just TOO umbrella like and it was too unsafe for the twinsSo we laid eyes on the Swift It was the double umbrella with a few bonuses that we were looking forIts sturdy that means the wheels wont spontaniously fall off like on the DuoGlider I loved how ea

0.8684897004587568

In [20]:
#b)
for word, coef in zip(vectorizer_2.get_feature_names_out(), model_2.coef_[0]):
    print(f"{word}: {coef}")


love: 1.3383847837945457
great: 0.93967223222708
easy: 1.152369990956628
old: 0.08785103331963807
little: 0.5078672782473773
perfect: 1.4903062859034046
loves: 1.7207345341004323
well: 0.5076393681322159
able: 0.20624679758998193
car: 0.07073736456151218
broke: -1.6436308138223066
less: -0.21066553788812137
even: -0.5110095270630157
waste: -2.008386237293198
disappointed: -2.428857415625606
work: -0.6244125969578616
product: -0.31729594412940504
money: -0.8991410073578493
would: -0.3400281012816716
return: -2.0650849635125548


In [21]:
#c)
import sys, time, timeit

print("model_2 score: {0}" .format(model_2.score(x_test_2, y_test_2)))
print("model score: {0}" .format(model.score(x_test, y_test)))

model_2.predict(x_test_2)
%time


model.predict(x_test)
%time

#hint: %time, %timeit

model_2 score: 0.8684897004587568
model score: 0.9318461215555756
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.58 µs
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.15 µs
